In [1]:
## pip install autox model which contains time series machine learning model
!pip install automl-x -i https://www.pypi.org/simple/

Looking in indexes: https://www.pypi.org/simple/


In [2]:
!pip install matplotlib

In [3]:
!pip install keras

In [4]:
!pip3 install tensorflow

In [5]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv('wtbdata_245days (1).csv')

def get_date(k):
    cur_date = "2020-01-01"
    one_day = timedelta(days=k-1)
    return str(datetime.strptime(cur_date, '%Y-%m-%d') + one_day)[:10]

df['Day'] = df['Day'].apply(lambda x: get_date(x))

def cols_concat(df, con_list):
    name = 't1'
    df[name] = df[con_list[0]].astype(str)
    for item in con_list[1:]:
        df[name] = df[name] + ' ' + df[item].astype(str)
    return df

df = cols_concat(df, ["Day", "Tmstamp"])

df = df[['TurbID', 't1', 'Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir', 'Pab1', 'Pab2', 'Pab3', 'Prtv', 'Patv']]
df['t1'] = pd.to_datetime(df['t1'])

In [7]:
df

,TurbID,t1,Wspd,Wdir,Etmp,Itmp,Ndir,Pab1,Pab2,Pab3,Prtv,Patv
0,1,2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2020-01-01 00:10:00,6.17,-3.99,30.73,41.80,25.92,1.00,1.00,1.00,-0.25,494.66
2,1,2020-01-01 00:20:00,6.27,-2.18,30.60,41.63,20.91,1.00,1.00,1.00,-0.24,509.76
3,1,2020-01-01 00:30:00,6.42,-0.73,30.52,41.52,20.91,1.00,1.00,1.00,-0.26,542.53
4,1,2020-01-01 00:40:00,6.25,0.89,30.49,41.38,20.91,1.00,1.00,1.00,-0.23,509.36
...,...,...,...,...,...,...,...,...,...,...,...,...
4727515,134,2020-09-01 23:10:00,7.79,2.80,-0.07,3.95,216.51,6.03,6.03,6.03,-111.69,601.44
4727516,134,2020-09-01 23:20:00,8.06,4.39,0.23,3.94,216.51,5.81,5.81,5.81,-72.67,673.79
4727517,134,2020-09-01 23:30:00,8.08,2.28,-0.16,4.15,216.51,0.68,0.68,0.68,-118.70,999.36
4727518,134,2020-09-01 23:40:00,8.46,0.80,-0.14,4.32,216.51,0.02,0.02,0.02,-58.12,1100.89


In [8]:
df_1 = df.head(100000)

In [9]:
from autox import AutoTS
autots = AutoTS(df = df_1,
                id_col = 'TurbID',
                time_col = 't1',
                target_col = 'Patv',
                time_varying_cols = ['Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir', 'Pab1', 'Pab2', 'Pab3', 'Prtv', 'Patv'],
                time_interval_num = 15,
                time_interval_unit = 'minute',
                forecast_period = 4*24*2)

In [ ]:
sub = autots.get_result()

   INFO ->  [+] feature engineer
   INFO ->  [+] fe_rolling_stat
100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.12s/it]
   INFO ->  [+] fe_lag
   INFO ->  [+] fe_diff
   INFO ->  [+] fe_time
   INFO ->  [+] feature combination
100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  4.62it/s]
   INFO ->  [+] construct data
 23%|██████████████████▌                                                              | 44/192 [03:40<22:06,  8.96s/it]   INFO ->  [+] sample data, frac=0.2604166666666667


 47%|█████████████████████████████████████▍                                         | 91/192 [26:51<1:34:13, 55.97s/it]   INFO ->  [+] sample data, frac=0.2604166666666667


 50%|███████████████████████████████████████▌                                       | 96/192 [32:11<1:40:51, 63.04s/it]   INFO ->  [+] sample data, frac=0.2604166666666667


In [ ]:
autots.feature_importances.head(10)

In [ ]:
sub.to_csv("./autox_kdd.csv", index=False)

In [ ]:
cur_TurbID = 1

plt.plot(df.loc[df['TurbID'] == cur_TurbID, 't1'], df.loc[df['TurbID'] == cur_TurbID, 'Patv'], color = 'b')
plt.plot(sub.loc[sub['TurbID'] == cur_TurbID, 't2'], sub.loc[sub['TurbID'] == cur_TurbID, 'y_mean'], color = 'r')

In [ ]:
cur_TurbID = 3

plt.plot(df.loc[df['TurbID'] == cur_TurbID, 't1'], df.loc[df['TurbID'] == cur_TurbID, 'Patv'], color = 'b')
plt.plot(sub.loc[sub['TurbID'] == cur_TurbID, 't2'], sub.loc[sub['TurbID'] == cur_TurbID, 'y_mean'], color = 'r')